# 目次
- はじめに
- pwd: 作業ディレクトリのパスを表示する
- ls: ファイルをリストする
- cd: 作業ディレクトリを変更する
- cp: ファイルをコピーする
- mv: ファイルを移動する
- rm: ファイルを消去する
- mkdir: ディレクトリを作る
- ファイル操作その他
- プロセスの実行

# はじめに
Pythonの標準ライブラリを用いると、シェルスクリプト（shやbash）と同等の操作がPython上で行えます。  
Pythonを用いることには次のような利点があります。
- Pythonの強力な文字列操作メソッドを用いられる
- シェルスクリプトの働きとPythonのその他のライブラリ（NumPyなど）とをシームレスにつなぐことができる  

特に後者は、Pythonの強みである「言語自体の汎用性とライブラリによる専門性」を表していると言えるでしょう。

本稿では、Pythonを用いてファイル操作を行う方法と、プロセスを実行する方法について、シェルのコマンドと対応させて簡単に紹介します。

# pwd: 作業ディレクトリのパスを表示する
pwdやlsなど、多くのファイル操作コマンドは、標準ライブラリ**os**モジュールを通して実現されています。  
現在の作業ディレクトリのパスはos.getcwd()によって得られ、文字列型として返されます。

In [1]:
import os
working_dir = os.getcwd()

# ls: ファイルをリストする
ls *path* は os.listdir(*path*) で、*path*におけるファイルなどを、文字列のリストとして返してくれます。

In [2]:
os.listdir('.')

['c02.csv',
 'Sample.gif',
 'c01.csv',
 'Pandas\u3000―\u3000データ分析.ipynb',
 'Pandas―データ分析（３）GroupBy.ipynb',
 'Seaborn ― matplotlib をより美しく、使いやすく.ipynb',
 'libsobol.c',
 'Fortran, C言語 との連携.ipynb',
 'RK4.f90',
 'Pandas―データ分析（５）応用：都道府県別人口推移.ipynb',
 'RK4_f2py.f90',
 'Pandas―データ分析（４）ファイル入出力.ipynb',
 'double_pendulum.gif',
 'SymPyー代数演算（２）応用：二重振り子.ipynb',
 'Pandas―データ分析（１）Series.ipynb',
 '.ipynb_checkpoints',
 'SymPyー代数演算（１）使い方.ipynb',
 'shやBashの代わりにPythonを使う.ipynb',
 'use_RK4.py',
 'SymPyー代数演算.ipynb',
 'Pandas―データ分析（２）DataFrame.ipynb',
 'Animation.ipynb']

os.listdirでは、任意の1文字を表す ? や任意の文字 \* などのワイルドカード、または正規表現を使うことができません。  
ワイルドカードや正規表現を用いて特定のファイル名のみをリストしたいときは、標準ライブラリの**glob**を用います。  
glob.glob(*path*)で、*path*に一致するファイルをリストとして抽出できます。ただし、os.listdirと異なり、ファイル名でなくそのパスを返していることに気をつけましょう。

In [3]:
import glob
ls2 = glob.glob("./*.ipynb")
for f in ls2:
    print(f)

./Pandas　―　データ分析.ipynb
./Pandas―データ分析（３）GroupBy.ipynb
./Seaborn ― matplotlib をより美しく、使いやすく.ipynb
./Fortran, C言語 との連携.ipynb
./Pandas―データ分析（５）応用：都道府県別人口推移.ipynb
./Pandas―データ分析（４）ファイル入出力.ipynb
./SymPyー代数演算（２）応用：二重振り子.ipynb
./Pandas―データ分析（１）Series.ipynb
./SymPyー代数演算（１）使い方.ipynb
./shやBashの代わりにPythonを使う.ipynb
./SymPyー代数演算.ipynb
./Pandas―データ分析（２）DataFrame.ipynb
./Animation.ipynb


# pathlib: 再帰的なファイル一覧
さらに高機能なPath操作をしたい場合は、**pathlib**ライブラリが便利です。  
**Path**オブジェクトでディレクトリを指定し、**glob**関数をワイルドカードとともに呼び出すと、サブディレクトリまで再帰的にファイルを検索します。Pathオブジェクトはイテレータになっているため、最初から全部のファイルを探すのではなく、forループごとに次のファイルへ進んでいきます。普通にリストが欲しい場合は、`list(path.glob('xxxx'))`を呼び出してください。

In [4]:
from pathlib import Path

# Pathオブジェクト(ここでは、一つ上のディレクトリを基準に作る)
path = Path('..')

#  再帰的にサブディレクトリ内の '.ipynb'ファイルを全て列挙する
for p in path.glob('**/*.ipynb'):
    print(p)

../ipynb2html.ipynb
../スライド/1日目スライド.ipynb
../応用編/Pandas　―　データ分析.ipynb
../応用編/Pandas―データ分析（３）GroupBy.ipynb
../応用編/Seaborn ― matplotlib をより美しく、使いやすく.ipynb
../応用編/Fortran, C言語 との連携.ipynb
../応用編/Pandas―データ分析（５）応用：都道府県別人口推移.ipynb
../応用編/Pandas―データ分析（４）ファイル入出力.ipynb
../応用編/SymPyー代数演算（２）応用：二重振り子.ipynb
../応用編/Pandas―データ分析（１）Series.ipynb
../応用編/SymPyー代数演算（１）使い方.ipynb
../応用編/shやBashの代わりにPythonを使う.ipynb
../応用編/SymPyー代数演算.ipynb
../応用編/Pandas―データ分析（２）DataFrame.ipynb
../応用編/Animation.ipynb
../応用編/.ipynb_checkpoints/shやBashの代わりにPythonを使う-checkpoint.ipynb
../.ipynb_checkpoints/ipynb2html-checkpoint.ipynb
../ライブラリ集/Sympy―代数演算.ipynb
../基礎編/Numpyを使う際の注意点.ipynb
../基礎編/Numpyの基礎（１）導入.ipynb
../基礎編/Numpyの基礎（７）便利なライブラリ群.ipynb
../基礎編/Matplotlib.ipynb
../基礎編/Numpyの基礎（４）データ型.ipynb
../基礎編/Numpyの基礎（６）ブロードキャスト.ipynb
../基礎編/Pythonの基礎2.ipynb
../基礎編/文字列操作（１）str.format.ipynb
../基礎編/Numpyの基礎（８）ファイル入出力.ipynb
../基礎編/Numpyの基礎（５）要素の参照.ipynb
../基礎編/numpy（基礎編）.ipynb
../基礎編/文字列操作（２）strやファイル読み込み.ipynb
../基礎編/Pythonの基礎1.ipynb
../

より詳細な情報は、[pathlib公式ドキュメント](https://docs.python.org/ja/3/library/pathlib.html)を参照してください。

# cd: 作業ディレクトリを変更する
cd *path* は os.chdir(*path*) です。  
Python上での作業ディレクトリの変更は、Pythonのセッションが終了するまで（IPythonなどを閉じるまで）有効です。

In [5]:
os.chdir("../基礎編")

# cp: ファイルをコピーする
cpやmvなどのコマンドは**shutil**モジュールによって提供されています。  
cp *src* *dst*  は shutil.copy(*src*, *dst*) です。

In [6]:
import shutil
print(glob.glob("./*.f90"))
shutil.copy("write_binary.f90", "copied_write_binary.f90")
print(glob.glob("./*.f90"))

['./write_binary.f90']
['./copied_write_binary.f90', './write_binary.f90']


また、ディレクトリまるごとのコピー（cp -r）は、 shutil.copytree(*src*, *dst*) です。

# mv: ファイルを移動する
mv *src* *dst* は shutil.move(*src*,*dst*) です。

In [7]:
print(glob.glob("./*.f90"))
shutil.move("copied_write_binary.f90", "moved_write_binary.f90")
print(glob.glob("./*.f90"))

['./copied_write_binary.f90', './write_binary.f90']
['./moved_write_binary.f90', './write_binary.f90']


# rm: ファイルを消去する
rm *path* は os.remove(*path*) です。

In [8]:
print(glob.glob("./*.f90"))
os.remove("moved_write_binary.f90")
print(glob.glob("./*.f90"))

['./moved_write_binary.f90', './write_binary.f90']
['./write_binary.f90']


また、空のディレクトリの消去（rm -r *dirname*）はos.rmdir(*dirname*)、  
中のファイルも含めたディレクトリまるごとの消去（rm -rf *dirname*）はshutil.rmtree(*dirname*)です。

# mkdir: ディレクトリを作る
mkdir *dirname* は os.mkdir(*dirname*) です。

# ファイル操作その他
osのサブモジュール**path**は、パスの操作に便利な関数を提供しています。

os.path.exists(*path*) *path*で表されるファイル・ディレクトリが存在していればTrue、存在していなければFalseを返す。  
os.path.isdir(*path*) *path*がディレクトリならばTrue、ファイルやシンボリックリンクならばFalseを返す。  
os.path.isfile(*path*) *path*がファイルならばTrue、ディレクトリやシンボリックリンクならばFalseを返す。

次の2つは、パスの文字列を場所+ファイル名に分割したり、拡張子を取り出すのに使えます。  
*dir*, *file* = os.path.split(*path*) *path*のファイル等の場所を*dir*に、ファイル名を*file*に文字列として返す。  
*root*, *ext* = os.path.splitext(*path*) *path*のファイル等の拡張子を*ext*に、拡張子の手前までを*root*に文字列として返す。

# プロセスの実行 - シェルを呼ぶ場合
上記のファイル操作コマンドだけでなく、Pythonからシェルを呼ぶことで任意のコマンドを実行することができます。
Python 3.5以降は**subprocess**モジュールのsubprocess.runを用いることが推奨されていますが、Python 2.xでも**os.system**から同様のことができます。

In [9]:
import subprocess
subprocess.run("./a.out > redirected.txt", shell=True) # Python 3.5以降
os.system("./a.out > redirected.txt") # Python 2.x, 現在は非推奨らしい。./a.outがないため正常終了の結果（0）を返さない

32512

subprocessモジュールからシェルを呼ばない場合や、標準出力・標準エラー出力を得る方法については、[こちら](https://docs.python.jp/3/library/subprocess.html)や[こちら](http://qiita.com/tdrk/items/9b23ad6a58ac4032bb3b)をご覧ください。

# IPythonマジックコマンド
Jupyter notebookやIPythonコンソールのみで使える非常に便利なマジックコマンドと呼ばれるものがあります。  
`%`から始まるコマンド、例えば`%pwd`や`%cd`などです。実は、有名なコマンドはだいたい`%`がなくても実行できちゃいます。しかも、変数への代入もできるのでコマンドの結果を使ったプログラムを書くことができます。

In [10]:
my_wd = %pwd

# 他には　%cd　や %mkdir などがある

# Jupyterからシェルのコマンドそのものを呼ぶ
Jupyterからシェルのコマンドそのものを呼ぶこともできます。  
`!` から始まる行は、`!`以降の文字列はJupyterが起動しているOSのシェルで実行されます。  
コマンド実行によって標準出力に吐かれるログを変数に代入ができます。

In [11]:
!which python

/usr/local/bin/python


ちなみに`which`コマンドはプログラムバイナリの位置を教えてくれるコマンドです。

In [12]:
python_location = !which python
python_location

['/usr/local/bin/python']

# 「!」コマンドにPython変数を埋め込む
シェルコマンドにPythonの変数を埋め込みたい時ありませんか？実はこれもできます。Python変数の前に`$`をつけることで、シェルの変数のように扱えます。

In [13]:
a = '基礎編'

In [14]:
!ls ../$a

Matplotlib.ipynb
NINO.3.csv
Numpyの基礎（３）ufunc.ipynb
Numpyの基礎（１）導入.ipynb
Numpyを使う際の注意点.ipynb
Numpyの基礎（２）生成関数.ipynb
Numpyの基礎（４）データ型.ipynb
Numpyの基礎（５）要素の参照.ipynb
Numpyの基礎（８）ファイル入出力.ipynb
Numpyの基礎（６）ブロードキャスト.ipynb
Numpyの基礎（７）便利なライブラリ群.ipynb
Pythonの基礎1.ipynb
Pythonの基礎2.ipynb
axis.png
c01.csv
ersst.v4.201702.nc
my_module.py
numpy（基礎編）.ipynb
redirected.txt
temp
write_binary.f90
ファイル読み込み.ipynb
文字列操作（１）str.format.ipynb
文字列操作（２）strやファイル読み込み.ipynb


このコマンドはforループの中でも使えます。

In [15]:
langs = ['python', 'ruby', 'java']
for lang in langs:
    path_bin = !which $lang
    print(path_bin)

['/usr/local/bin/python']
['/usr/bin/ruby']
['/usr/bin/java']


ただし、`!`を使ったシェルの呼び出しはあなたの環境でしか動かない可能性があるので、他の人にプログラムを渡す場合は使ってはいけません。  
**os**などの標準ライブラリは環境依存しないように作られているので、こちらを使うようにしましょう。